In [1]:
# Import necessary libraries
import requests  # for making HTTP requests to fetch the webpage content
from bs4 import BeautifulSoup  # for parsing HTML and extracting data from it
import pandas as pd  # for working with data and creating a DataFrame

In [2]:
# Set the URL of the website to scrape
url = "https://quotes.toscrape.com/page/<placeholder>/"
# Send a GET request to the URL and store the response
response = requests.get(url)

In [3]:
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
# Initialize an empty list to store DataFrames from multiple pages
df_list = []


# Base URL pattern for accessing different pages of the book listings
base_url = "https://quotes.toscrape.com/page/<placeholder>/"
base_url1 = "https://quotes.toscrape.com/<placeholder>/" #this url is only for going to born and description pages as its url is different

# Loop through pages 1 to 10
for page in range(1, 11):
    # Replace '<placeholder>' with the actual page number to create the full URL
    page_url = base_url.replace("<placeholder>", str(page))

    # Send a GET request to the current page URL
    response = requests.get(page_url)

    # Parse the HTML content of the current page
    soup = BeautifulSoup(response.content, 'html.parser')

    div_tags=soup.find_all('div',class_='quote')


    # Initialize an empty list to store book details for the current page
    all_list = []

    # Loop through all books found on the current page
    for tag in div_tags:
        # Extract rating, title, price, and stock status for each book
        quote = tag.find("span", class_="text").text
        span_tag=tag.find('span',class_=None)
        author=span_tag.find('small',class_='author').text
        name_tag=tag.find('div',class_='tags').meta['content']

        author_url = base_url1.replace("<placeholder>", tag.find("a")["href"])
        author_response = requests.get(author_url)
        author_soup = BeautifulSoup(author_response.text, 'html.parser')

        birth_date = author_soup.find("span", class_="author-born-date").text
        birth_location = author_soup.find("span", class_="author-born-location").text
        description = author_soup.find("div", class_="author-description").get_text().strip()

        # Append the book details as a tuple to the list
        all_list.append((quote, author, name_tag, birth_date, birth_location, description))

    # Create a DataFrame using the list of book details for the current page
    df = pd.DataFrame(all_list, columns=["Quote", "Author Name Surname", "Tags", "Birth Date", "Birth Location", "Author Description"])

    # Append the DataFrame to the list of DataFrames for each page
    df_list.append(df)


In [5]:
final_df=pd.concat(df_list, ignore_index=True)


In [6]:
final_df

,Quote,Author NameSurname,Tags,Birth Date,Birth Location,Author Description
0,“The world as we have created it is a process ...,Albert Einstein,"change,deep-thoughts,thinking,world","March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in Ulm, Germ..."
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities,choices","July 31, 1965","in Yate, South Gloucestershire, England, The U...",See also: Robert GalbraithAlthough she writes ...
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational,life,live,miracle,miracles","March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in Ulm, Germ..."
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy,books,classic,humor","December 16, 1775","in Steventon Rectory, Hampshire, The United Ki...",Jane Austen was an English novelist whose work...
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"be-yourself,inspirational","June 01, 1926",in The United States,Marilyn Monroe (born Norma Jeane Mortenson; Ju...
...,...,...,...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,better-life-empathy,"April 28, 1926","in Monroeville, Alabama, The United States","Harper Lee, known as Nelle, was born in the Al..."
96,“You have to write the book that wants to be w...,Madeleine L'Engle,"books,children,difficult,grown-ups,write,write...","November 29, 1918","in New York City, New York, The United States",Madeleine L'Engle was an American writer best ...
97,“Never tell the truth to people who are not wo...,Mark Twain,truth,"November 30, 1835","in Florida, Missouri, The United States","Samuel Langhorne Clemens, better known by his ..."
98,"“A person's a person, no matter how small.”",Dr. Seuss,inspirational,"March 02, 1904","in Springfield, MA, The United States",Theodor Seuss Geisel was born 2 March 1904 in ...
